In [1]:
#jupyter上にて
#python側より html-javascriptのデータを取得
#html-javascript側よりpython側のデータを取得

In [2]:
%%html
<!DOCTYPE html><html lang="ja">
<head><mata charset="utf-8"/><title>widget test</title></head><body>

<form name="form1">
送信テキスト：
<input type="text" name="input1" value="フロント側テキスト「abc,def」" style="width:300px;"><br>
ログ：<br>
<textarea disabled name="output1" rows="3" cols="60"></textarea><br>
<button name="btn1" type="button">受信ボタン</button>
</form>

<script>
var ipy_TransView = ""

document.form1.btn1.onclick = function() {
    //b2f_order_textを変化させる(+1する)、バック側のコールバック関数を起こし、バック側のテキストを受信                                      
    ipy_TransView.model.set('b2f_order_text', (ipy_TransView.model.get('b2f_order_text')+1))
    ipy_TransView.touch();
};

//ウィジェットで値を取得
require.undef('trans');
define('trans', ["@jupyter-widgets/base"], function(widgets) {
    var TransView = widgets.DOMWidgetView.extend({
        render: function() {
            //コールバック関数をセット
            this.model.on('change:f2b_order_text', this.f2b_order_text_changed, this);
            this.model.on('change:b2f_mess_text', this.b2f_mess_text_changed, this);
            
            ipy_TransView = this //外側からウィジェットにアクセスできるようにセット
        },
        f2b_order_text_changed: function() {//バック側からテキスト送信指令が来たときのコールバック
            this.model.set('f2b_mess_text', document.form1.input1.value);//送信するテキストをセット              
            this.touch();
        },
        b2f_mess_text_changed: function() {//バック側からテキストを受信したときのコールバック            
            document.form1.output1.value += 'バック側からテキストを受信：\n'
            document.form1.output1.value += this.model.get('b2f_mess_text')+'\n'//受信テキストを表示       
            this.touch();
        },

    });
    return {
        TransView : TransView
    };
});

</script></body></html>

In [3]:
#取得用のウイジェット作成
import ipywidgets as widgets
from traitlets import Unicode,Int
class TransWidget(widgets.DOMWidget):
    _view_name = Unicode('TransView').tag(sync=True)
    _view_module = Unicode('trans').tag(sync=True)
    f2b_order_text = Int(0).tag(sync=True) #バック側より、フロント⇒バック側へテキストを送る指令用
    f2b_mess_text = Unicode('').tag(sync=True) #フロント⇒バック側へ送るテキストの中身
    b2f_order_text = Int(0).tag(sync=True) #フロント側より、バック⇒フロント側へテキストを送る指令用
    b2f_mess_text = Unicode('').tag(sync=True) #バック⇒フロント側へ送るテキストの中身
trans = TransWidget()

def b2f_order_text_changed(change):#フロント側からテキスト送信指令が来たときのコールバック
    trans.b2f_mess_text = back_text#送信するテキストをセット  

def f2b_mess_text_changed(change):#フロント側からテキストを受信したときのコールバック
    print('フロント側よりテキストを受信：')
    print(trans.f2b_mess_text)#受信テキストを表示

#コールバック関数をセット
trans.observe(b2f_order_text_changed, names='b2f_order_text')
trans.observe(f2b_mess_text_changed, names='f2b_mess_text')

#バック側のテキスト内容をセット"
back_text = "バック側テキスト「ghi,jkl」"

trans #ここで一度ウイジェットを作る必要がある

TransWidget()

フロント側よりテキストを受信：
フロント側テキスト「abc,def」


In [4]:
#f2b_order_textを変化させる(+1する)ことで、フロント側のコールバック関数を起こし、フロント側のテキストを受信
trans.f2b_order_text += 1 